In [2]:
import keras
import cv2
import numpy as np

from scipy.io import loadmat
from sklearn.cluster import DBSCAN

In [3]:
model_path = 'best_model.h5'
model = keras.models.load_model(model_path)

# Load your MAT file
data = loadmat('SC2.mat')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [4]:
width, height = 1280, 720

locations = np.stack((data['Top'], data['Bottom'], data['Left'], data['Right']), axis=-1)
locations += 1

# 변환 적용
locations[:, :, 2] = (locations[:, :, 2] * width / 2)
locations[:, :, 3] = (locations[:, :, 3] * width / 2)
locations[:, :, 1] = height - (locations[:, :, 1] * height / 2)
locations[:, :, 0] = height - (locations[:, :, 0] * height / 2)

# 모든 요소를 정수로 변환
locations = locations.astype(int)

In [5]:
def preprocess_image(image):
    # 이미지 크롭
    x, y, width, height = 570, 220, 210, 180
    cropped_image = image[y:y+height, x:x+width]

    # 흑백으로 변환
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    # 리사이징
    resized_image = cv2.resize(gray_image, (128, 128))
    
    # 모델에 적용할 수 있도록 차원 확장
    processed_image = np.expand_dims(resized_image, axis=0)
    
    return processed_image

In [6]:
def check_crossing(bounding_boxes, width, height):
    """
    객체가 특정 선을 넘었는지 확인하는 함수.

    Parameters:
    bounding_boxes (list of tuples): 바운딩 박스 리스트, 각 바운딩 박스는 (bottom, top, left, right) 형태
    line_position (float): 검사할 선의 위치 (이미지 높이에 대한 비율)

    Returns:
    list: 선을 넘은 바운딩 박스의 인덱스 리스트
    """
    
    line = [311, [570, 780]]
    crossing_indexes = []

    for index, (bottom, top, left, right) in enumerate(bounding_boxes):
        # 객체가 선을 넘었는지 확인 (여기서는 수평선을 기준으로 함)
        if [bottom, top, left, right] == [360, 360, 640, 640]:
            continue
        if (bottom <= line[0] and top >= line[0]) and not (left > line[1][0] and right < line[1][1]):
            crossing_indexes.append(index)

    return crossing_indexes


In [7]:
def cluster_bounding_boxes(bounding_boxes, eps):
    min_samples = 2
    space_center = [675, 310]
    space_size = [210, 180]

    # [0,0,0,0]이 아닌 바운딩 박스만 필터링
    valid_boxes = np.array([bbox for bbox in bounding_boxes if not np.array_equal(bbox, [360, 360, 640, 640])])

    # 필터링된 바운딩 박스의 인덱스를 추적
    valid_indices = np.array([i for i, bbox in enumerate(bounding_boxes) if not np.array_equal(bbox, [360, 360, 640, 640])])

    # 바운딩 박스의 중심점 계산
    centers = np.array([((bbox[2] + bbox[3]) / 2, (bbox[0] + bbox[1]) / 2) for bbox in valid_boxes])

    # 특정 공간 내 바운딩 박스 필터링
    space_min = np.array(space_center) - np.array(space_size) / 2
    space_max = np.array(space_center) + np.array(space_size) / 2
    in_space_mask = (centers >= space_min) & (centers <= space_max)
    filtered_indices = np.where(in_space_mask.all(axis=1))[0]

    # 실제 필터링된 바운딩 박스에 대한 중심점 사용
    filtered_centers = centers[filtered_indices]
    # 필터링된 바운딩 박스의 원래 인덱스 매핑
    valid_filtered_indices = valid_indices[filtered_indices]

    # DBSCAN 클러스터링
    if len(filtered_centers) > 0:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        cluster_labels = dbscan.fit_predict(filtered_centers)

        # 클러스터에 속한 바운딩 박스의 원래 인덱스 반환
        clustered_indices = [valid_filtered_indices[cluster_labels == i] for i in range(max(cluster_labels) + 1) if i != -1]
    else:
        clustered_indices = []

    return clustered_indices

In [26]:
def is_parked(bounding_boxes):
    """
    주차 공간에 차량(바운딩 박스)이 있는지 확인하는 함수.

    :param bounding_boxes: 바운딩 박스 목록, 각 바운딩 박스는 [top, bottom, left, right] 형태
    :param parking_area: 주차 공간을 정의하는 바운딩 박스, [top, bottom, left, right] 형태
    :return: 주차 공간 내 차량 존재 여부, True 또는 False
    """
    # 주차 공간의 중심 계산
    parking_area = [355, 383, 101, 157]
    parking_center = [(parking_area[2] + parking_area[3]) / 2, (parking_area[0] + parking_area[1]) / 2]

    # 각 바운딩 박스의 중심 계산 및 주차 공간 내 존재 여부 확인
    for bbox in bounding_boxes:
        center = [(bbox[2] + bbox[3]) / 2, (bbox[0] + bbox[1]) / 2]
        if parking_area[2] <= center[0] <= parking_area[3] and parking_area[0] <= center[1] <= parking_area[1]:
            return True

    return False  # 주차 공간 내에 바운딩 박스 없음

In [30]:
# 비디오 코덱 설정
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# 비디오 라이터 객체 생성
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1280, 720))
# 비디오 캡처 객체 생성
video_path = 'test1.mp4'
cap = cv2.VideoCapture(video_path) # 웹캠이나 비디오 파일의 경로를 입력할 수 있습니다. 0은 기본 웹캠을 의미합니다.
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
i = 0
cnt = 0
while True:
    # 프레임 읽기
    ret, frame = cap.read()
    
    # 프레임이 제대로 읽혔는지 확인
    if not ret:
        break
    now_location = locations[i]

    if is_parked(now_location):
        cnt += 1
        print(cnt)
        if cnt > 100:
            [bottom, top, left, right] = [355, 383, 101, 150]
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, 'illegal parking', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
    
    processed_image = preprocess_image(frame)

    processed_image_expanded = np.expand_dims(processed_image, axis=-1)
    # 모델 적용
    predicted_probs = model.predict(processed_image_expanded)[0]
    predicted_class = np.argmax(predicted_probs)
    if predicted_class== 4:
        result = 'No'
    else:
        result = 'Yes'
    # 결과 표시
    if result == 'Yes':
        tail_indexs = cluster_bounding_boxes(now_location, 50)
        if tail_indexs:  # tail_indexs 리스트에 값이 있을 경우에만 반복 실행
            for index in tail_indexs[0]:  # 첫 번째 클러스터의 인덱스를 반복 처리
                [bottom, top, left, right] = now_location[index]
                # 영상에 사각형 그리기 (BGR 색상 코드: 파란색, 선 두께: 2)
                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                cv2.putText(frame, 'illegal tail bite', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
        else:
            print("반환된 클러스터가 없습니다.")
    crossing_indexs = check_crossing(now_location, width, height)
    for index in crossing_indexs:
        [bottom, top, left, right] = now_location[index]

        # 영상에 사각형 그리기 (BGR 색상 코드: 빨간색, 선 두께: 2)
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, 'illegal u-turn', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.putText(frame, f'illegal tail bite: {result}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # 결과를 화면에 표시
    
    cv2.imshow('Video', frame)  # 비디오 프레임을 화면에 표시
    
    # 처리된 프레임을 비디오 파일에 쓰기
    out.write(frame)

    i += 1
    # ESC 키를 눌러 종료
    if cv2.waitKey(1) == 27:
        break

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()

반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
반환된 클러스터가 없습니다.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
반환된 클러스터가 없습니다.
74
반환된 클러스터가 없습니다.
75
반환된 클러스터가 없습니다.
76
반환된 클러스터가 없습니다.
77
반환된 클러스터가 없습니다.
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
반환된 클러스터가 없습니다.
134
반환된 클러스터가 없습니다.
135
반환된 클러스터가 없습니다.
136
반환된 클러스터가 없습니다.
137
반환된 클러스터가 없습니다.
138
반환된 클러스터가 없습니다.
139
140
141
142
143
반환된 클러스터가 없습니다.
144
반환된 클러스터가 없습니다.
145
반환된 클러스터가 없습니다.
146
반환된 클러스터가 없습니다.
147
반환된 클러스터가 없습니다.
148
반환된 클러스터가 없습니다.
149
반환된 클러스터가 없습니다.
150
151
152
153


In [33]:
# # 비디오 코덱 설정
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# # 비디오 라이터 객체 생성
# out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1280, 720))
# # 비디오 캡처 객체 생성
# video_path = 'test1.mp4'
# cap = cv2.VideoCapture(video_path) # 웹캠이나 비디오 파일의 경로를 입력할 수 있습니다. 0은 기본 웹캠을 의미합니다.
# width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
# height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
# i = 0
# cnt = 0
# while True:
#     # 프레임 읽기
#     ret, frame = cap.read()
    
#     # 프레임이 제대로 읽혔는지 확인
#     if not ret:
#         break
#     now_location = locations[i]

#     if is_parked(now_location):
#         cnt += 1
#         if cnt > 100:
#             [bottom, top, left, right] = [355, 383, 101, 150]
#             cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
#             cv2.putText(frame, 'illegal parking', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
    
#     processed_image = preprocess_image(frame)

#     processed_image_expanded = np.expand_dims(processed_image, axis=-1)
#     # 모델 적용
#     predicted_probs = model.predict(processed_image_expanded)[0]
#     predicted_class = np.argmax(predicted_probs)

#     tail_indexs = cluster_bounding_boxes(now_location, 50)
#     if tail_indexs:  # tail_indexs 리스트에 값이 있을 경우에만 반복 실행
#         for index in tail_indexs[0]:  # 첫 번째 클러스터의 인덱스를 반복 처리
#             [bottom, top, left, right] = now_location[index]
#             # 영상에 사각형 그리기 (BGR 색상 코드: 파란색, 선 두께: 2)
#             cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
#             cv2.putText(frame, 'illegal tail bite', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

#     crossing_indexs = check_crossing(now_location, width, height)
#     for index in crossing_indexs:
#         [bottom, top, left, right] = now_location[index]

#         # 영상에 사각형 그리기 (BGR 색상 코드: 빨간색, 선 두께: 2)
#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
#         cv2.putText(frame, 'illegal u-turn', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
    
#     cv2.imshow('Video', frame)  # 비디오 프레임을 화면에 표시
    
#     # 처리된 프레임을 비디오 파일에 쓰기
#     out.write(frame)

#     i += 1
#     # ESC 키를 눌러 종료
#     if cv2.waitKey(1) == 27:
#         break

# # 자원 해제
# cap.release()
# out.release()
# cv2.destroyAllWindows()